In [1]:
import pandas as pd
import matplotlib.pyplot as plt

INPUT SYMBOL

In [2]:
symbol = input("Inserte el ticker de la empresa: ").upper()

CARGA DE EXCELS

In [ ]:
print(f"Suba los archivos {symbol}_INCOME_STATEMENT.xlsx, {symbol}_BALANCE_SHEET.xlsx y f'{symbol}_CASH_FLOW.xlsx' modificados (como se solicita en DCF.ipynb) y {symbol}_precios.xlsx sin modificar")

# Convertir los excel a DataFrames
balance_sheet_df = pd.read_excel(f'{symbol}_BALANCE_SHEET.xlsx')
income_statement_df = pd.read_excel(f'{symbol}_INCOME_STATEMENT.xlsx')
cash_flow_df = pd.read_excel(f'{symbol}_CASH_FLOW.xlsx')
prices_df = pd.read_excel(f'{symbol}_precios.xlsx')

COCIENTE CORRIENTE

In [4]:
# Buscar las filas correspondientes a 'totalCurrentAssets' y 'totalCurrentLiabilities' en la segunda columna
total_current_assets = balance_sheet_df[balance_sheet_df.iloc[:, 0] == 'totalCurrentAssets']
total_current_liabilities = balance_sheet_df[balance_sheet_df.iloc[:, 0] == 'totalCurrentLiabilities']

# Asegurarse de que se encontró una única fila para cada uno
if total_current_assets.shape[0] != 1 or total_current_liabilities.shape[0] != 1:
    raise ValueError("No se encontraron filas únicas para 'totalCurrentAssets' o 'totalCurrentLiabilities'")

# Obtener los valores excluyendo la primera columna (que contiene el nombre de las filas)
total_current_assets_values = total_current_assets.iloc[0, 1:].astype(float)
total_current_liabilities_values = total_current_liabilities.iloc[0, 1:].astype(float)

# Calcular el Cociente Corriente
Cociente_corriente = (total_current_liabilities_values / total_current_assets_values) * 100

# Crear un nuevo DataFrame para los ratios de liquidez en formato de porcentaje
ratios_liquidez_df = pd.DataFrame({
    'Fecha': total_current_assets.columns[1:],  # Excluyendo la primera columna
    'Cociente Corriente': Cociente_corriente
})

# Convertir los valores a formato de porcentaje
ratios_liquidez_df['Cociente Corriente'] = ratios_liquidez_df['Cociente Corriente'].map("{:.2f}%".format)

# Ajustar el índice del DataFrame para evitar duplicación de la columna Fecha
ratios_liquidez_df.set_index('Fecha', inplace=True)

RELACION DE PRUEBA ACIDA

In [5]:
# Buscar las filas correspondientes a 'totalCurrentAssets', 'inventory' y 'totalCurrentLiabilities' en la primera columna
total_current_assets = balance_sheet_df[balance_sheet_df.iloc[:, 0] == 'totalCurrentAssets']
inventory = balance_sheet_df[balance_sheet_df.iloc[:, 0] == 'inventory']
total_current_liabilities = balance_sheet_df[balance_sheet_df.iloc[:, 0] == 'totalCurrentLiabilities']

# Asegurarse de que se encontró una única fila para cada uno
if total_current_assets.shape[0] != 1 or inventory.shape[0] != 1 or total_current_liabilities.shape[0] != 1:
    raise ValueError("No se encontraron filas únicas para 'totalCurrentAssets', 'inventory' o 'totalCurrentLiabilities'")

# Obtener los valores excluyendo la primera columna (que contiene el nombre de las filas)
total_current_assets_values = total_current_assets.iloc[0, 1:].astype(float)
inventory_values = inventory.iloc[0, 1:].astype(float)
total_current_liabilities_values = total_current_liabilities.iloc[0, 1:].astype(float)

# Calcular la Relación de Prueba Ácida
relacion_prueba_acida = (total_current_assets_values - inventory_values) / total_current_liabilities_values

# Crear un DataFrame para la fecha y la Relación de Prueba Ácida
relacion_prueba_acida_df = pd.DataFrame({
    'Fecha': total_current_assets.columns[1:],  # Excluyendo la primera columna
    'Relación de Prueba Ácida': relacion_prueba_acida
})

# Agregar la columna al DataFrame ratios_liquidez_df
ratios_liquidez_df['Relación de Prueba Ácida'] = relacion_prueba_acida_df['Relación de Prueba Ácida'].values

COEFICIENTE DE EFECTIVO

In [6]:
# Buscar las filas correspondientes a 'cashAndShortTermInvestments' y 'totalCurrentLiabilities' en la primera columna
cash_and_short_term_investments = balance_sheet_df[balance_sheet_df.iloc[:, 0] == 'cashAndShortTermInvestments']
total_current_liabilities = balance_sheet_df[balance_sheet_df.iloc[:, 0] == 'totalCurrentLiabilities']

# Asegurarse de que se encontró una única fila para cada uno
if cash_and_short_term_investments.shape[0] != 1 or total_current_liabilities.shape[0] != 1:
    raise ValueError("No se encontraron filas únicas para 'cashAndShortTermInvestments' o 'totalCurrentLiabilities'")

# Obtener los valores excluyendo la primera columna (que contiene el nombre de las filas)
cash_and_short_term_investments_values = cash_and_short_term_investments.iloc[0, 1:].astype(float)
total_current_liabilities_values = total_current_liabilities.iloc[0, 1:].astype(float)

# Calcular el Coeficiente de Efectivo en porcentaje
coeficiente_efectivo = (cash_and_short_term_investments_values / total_current_liabilities_values) * 100

# Crear un DataFrame para la fecha y el Coeficiente de Efectivo
coeficiente_efectivo_df = pd.DataFrame({
    'Fecha': cash_and_short_term_investments.columns[1:],  # Excluyendo la primera columna
    'Coeficiente de Efectivo': coeficiente_efectivo
})

# Convertir los valores a formato de porcentaje
coeficiente_efectivo_df['Coeficiente de Efectivo'] = coeficiente_efectivo_df['Coeficiente de Efectivo'].map("{:.2f}%".format)

# Agregar la columna al DataFrame ratios_liquidez_df
ratios_liquidez_df['Coeficiente de Efectivo'] = coeficiente_efectivo_df['Coeficiente de Efectivo'].values

RATIO DE FLUJO DE CAJA OPERATIVO

In [7]:
# Buscar las filas correspondientes a 'operatingCashflow' en cash_flow_df y 'totalCurrentLiabilities' en balance_sheet_df
operating_cashflow = cash_flow_df[cash_flow_df.iloc[:, 0] == 'operatingCashflow']
total_current_liabilities = balance_sheet_df[balance_sheet_df.iloc[:, 0] == 'totalCurrentLiabilities']

# Asegurarse de que se encontró una única fila para cada uno
if operating_cashflow.shape[0] != 1 or total_current_liabilities.shape[0] != 1:
    raise ValueError("No se encontraron filas únicas para 'operatingCashflow' o 'totalCurrentLiabilities'")

# Obtener los valores excluyendo la primera columna (que contiene el nombre de las filas)
operating_cashflow_values = operating_cashflow.iloc[0, 1:].astype(float)
total_current_liabilities_values = total_current_liabilities.iloc[0, 1:].astype(float)

# Calcular el Ratio de Flujo de Caja Operativo en porcentaje
ratio_flujo_caja_operativo = (operating_cashflow_values / total_current_liabilities_values) * 100

# Crear un DataFrame para la fecha y el Ratio de Flujo de Caja Operativo
ratio_flujo_caja_operativo_df = pd.DataFrame({
    'Fecha': operating_cashflow.columns[1:],  # Excluyendo la primera columna
    'Ratio de Flujo de Caja Operativo': ratio_flujo_caja_operativo
})

# Convertir los valores a formato de porcentaje
ratio_flujo_caja_operativo_df['Ratio de Flujo de Caja Operativo'] = ratio_flujo_caja_operativo_df['Ratio de Flujo de Caja Operativo'].map("{:.2f}%".format)

# Agregar la columna al DataFrame ratios_liquidez_df
ratios_liquidez_df['Ratio de Flujo de Caja Operativo'] = ratio_flujo_caja_operativo_df['Ratio de Flujo de Caja Operativo'].values

***RATIOS DE LIQUIDEZ***

In [ ]:
# Mostrar los resultados
print("DataFrame con Ratios de Liquidez por Año:")
print(ratios_liquidez_df)

In [ ]:
# Convertir las columnas de porcentaje a valores numéricos, manejando excepciones
def convert_percentage(x):
    try:
        return float(x.strip('%'))
    except AttributeError:
        return x

ratios_liquidez_df = ratios_liquidez_df.applymap(convert_percentage)

# Graficar cada columna del DataFrame
for column in ratios_liquidez_df.columns:
    plt.figure(figsize=(10, 6))
    plt.plot(ratios_liquidez_df.index, ratios_liquidez_df[column], marker='o', linestyle='-', color='b')
    plt.title(f'{column} por Año')
    plt.xlabel('Fecha')
    plt.ylabel(column)
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.tight_layout()
    plt.show()

RATIO DE ENDEUDAMIENTO

In [10]:
# Buscar las filas correspondientes a 'totalLiabilities' y 'totalAssets' en la primera columna
total_liabilities = balance_sheet_df[balance_sheet_df.iloc[:, 0] == 'totalLiabilities']
total_assets = balance_sheet_df[balance_sheet_df.iloc[:, 0] == 'totalAssets']

# Asegurarse de que se encontró una única fila para cada uno
if total_liabilities.shape[0] != 1 or total_assets.shape[0] != 1:
    raise ValueError("No se encontraron filas únicas para 'totalLiabilities' o 'totalAssets'")

# Obtener los valores excluyendo la primera columna (que contiene el nombre de las filas)
total_liabilities_values = total_liabilities.iloc[0, 1:].astype(float)
total_assets_values = total_assets.iloc[0, 1:].astype(float)

# Calcular el Ratio de Endeudamiento en porcentaje
ratio_endeudamiento = (total_liabilities_values / total_assets_values) * 100

# Crear un DataFrame para la fecha y el Ratio de Endeudamiento
apalancamiento_df = pd.DataFrame({
    'Fecha': total_liabilities.columns[1:],  # Excluyendo la primera columna
    'Ratio de Endeudamiento': ratio_endeudamiento
})

# Convertir los valores a formato de porcentaje
apalancamiento_df['Ratio de Endeudamiento'] = apalancamiento_df['Ratio de Endeudamiento'].map("{:.2f}%".format)

# Establecer la columna de fechas como índice para evitar duplicación
apalancamiento_df.set_index('Fecha', inplace=True)

RATIO DEUDA/CAPITAL

In [11]:
# Buscar las filas correspondientes a 'totalLiabilities' y 'totalShareholderEquity' en la primera columna
total_liabilities = balance_sheet_df[balance_sheet_df.iloc[:, 0] == 'totalLiabilities']
total_shareholder_equity = balance_sheet_df[balance_sheet_df.iloc[:, 0] == 'totalShareholderEquity']

# Asegurarse de que se encontró una única fila para cada uno
if total_liabilities.shape[0] != 1 or total_shareholder_equity.shape[0] != 1:
    raise ValueError("No se encontraron filas únicas para 'totalLiabilities' o 'totalShareholderEquity'")

# Obtener los valores excluyendo la primera columna (que contiene el nombre de las filas)
total_liabilities_values = total_liabilities.iloc[0, 1:].astype(float)
total_shareholder_equity_values = total_shareholder_equity.iloc[0, 1:].astype(float)

# Calcular el Ratio de Deuda a Capital
ratio_deuda_capital = (total_liabilities_values / total_shareholder_equity_values) * 100

# Crear un DataFrame para la fecha y el Ratio de Deuda a Capital
ratio_deuda_capital_df = pd.DataFrame({
    'Fecha': total_liabilities.columns[1:],  # Excluyendo la primera columna
    'Ratio de Deuda a Capital': ratio_deuda_capital
})

# Convertir la columna 'Fecha' a tipo datetime
ratio_deuda_capital_df['Fecha'] = pd.to_datetime(ratio_deuda_capital_df['Fecha'])

# Establecer 'Fecha' como índice en ratio_deuda_capital_df
ratio_deuda_capital_df.set_index('Fecha', inplace=True)

# Si 'Fecha' está en las columnas de apalancamiento_df, eliminarla para evitar duplicaciones
if 'Fecha' in apalancamiento_df.columns:
    apalancamiento_df.drop(columns=['Fecha'], inplace=True)

# Asegurarse de que el índice de apalancamiento_df sea de tipo datetime para la columna 'Fecha'
if not pd.api.types.is_datetime64_any_dtype(apalancamiento_df.index):
    apalancamiento_df.index = pd.to_datetime(apalancamiento_df.index)

# Agregar la columna al DataFrame apalancamiento_df
apalancamiento_df['Ratio de Deuda a Capital'] = ratio_deuda_capital_df['Ratio de Deuda a Capital']

RATIO EBIT/INTERESES

In [12]:
# Buscar las filas correspondientes a 'ebit' y 'interestExpense' en la primera columna
ebit = income_statement_df[income_statement_df.iloc[:, 0] == 'ebit']
interest_expense = income_statement_df[income_statement_df.iloc[:, 0] == 'interestExpense']

# Asegurarse de que se encontró una única fila para cada uno
if ebit.shape[0] != 1 or interest_expense.shape[0] != 1:
    raise ValueError("No se encontraron filas únicas para 'ebit' o 'interestExpense'")

# Obtener los valores excluyendo la primera columna (que contiene el nombre de las filas)
ebit_values = ebit.iloc[0, 1:].astype(float).values
interest_expense_values = interest_expense.iloc[0, 1:].astype(float).values

# Calcular el RATIO EBIT/INTERESES
ratio_ebit_intereses = ebit_values / interest_expense_values

# Crear un DataFrame para la fecha y el RATIO EBIT/INTERESES
ratio_ebit_intereses_df = pd.DataFrame({
    'Fecha': ebit.columns[1:],  # Excluyendo la primera columna
    'RATIO EBIT/INTERESES': ratio_ebit_intereses
})

# Convertir la columna 'Fecha' a tipo datetime
ratio_ebit_intereses_df['Fecha'] = pd.to_datetime(ratio_ebit_intereses_df['Fecha'])

# Establecer 'Fecha' como índice en ratio_ebit_intereses_df
ratio_ebit_intereses_df.set_index('Fecha', inplace=True)

# Si 'Fecha' está en las columnas de apalancamiento_df, eliminarla para evitar duplicaciones
if 'Fecha' in apalancamiento_df.columns:
    apalancamiento_df.drop(columns=['Fecha'], inplace=True)

# Asegurarse de que el índice de apalancamiento_df sea de tipo datetime para la columna 'Fecha'
if not pd.api.types.is_datetime64_any_dtype(apalancamiento_df.index):
    apalancamiento_df.index = pd.to_datetime(apalancamiento_df.index)

# Agregar la columna al DataFrame apalancamiento_df
apalancamiento_df['RATIO EBIT/INTERESES'] = ratio_ebit_intereses_df['RATIO EBIT/INTERESES']

***RATIOS DE APALANCAMIENTO***

In [ ]:
print("DataFrame con Ratios de Apalancamiento por Año:")
print(apalancamiento_df)

In [ ]:
# Verificar si 'Fecha' está en las columnas de apalancamiento_df
if 'Fecha' in apalancamiento_df.columns:
    # Convertir la columna 'Fecha' a tipo datetime si no lo está
    apalancamiento_df['Fecha'] = pd.to_datetime(apalancamiento_df['Fecha'])
    # Establecer 'Fecha' como índice
    apalancamiento_df.set_index('Fecha', inplace=True)

# Convertir las columnas de porcentaje a float para ordenarlas correctamente, si no están ya en float
if apalancamiento_df['Ratio de Endeudamiento'].dtype == 'object':
    apalancamiento_df['Ratio de Endeudamiento'] = apalancamiento_df['Ratio de Endeudamiento'].str.rstrip('%').astype('float') / 100
if apalancamiento_df['Ratio de Deuda a Capital'].dtype == 'object':
    apalancamiento_df['Ratio de Deuda a Capital'] = apalancamiento_df['Ratio de Deuda a Capital'].str.rstrip('%').astype('float') / 100

# Crear gráficos para cada ratio en el DataFrame apalancamiento_df
fig, axs = plt.subplots(len(apalancamiento_df.columns), 1, figsize=(10, 15))

for i, column in enumerate(apalancamiento_df.columns):
    axs[i].plot(apalancamiento_df.index, apalancamiento_df[column], marker='o')
    axs[i].set_title(column)
    axs[i].set_xlabel('Fecha')
    axs[i].set_ylabel(column)
    axs[i].grid(True)

plt.tight_layout()
plt.show()

MARGEN BRUTO

In [15]:
# Buscar las filas correspondientes a 'grossProfit' y 'totalRevenue' en la primera columna
gross_profit = income_statement_df[income_statement_df.iloc[:, 0] == 'grossProfit']
total_revenue = income_statement_df[income_statement_df.iloc[:, 0] == 'totalRevenue']

# Asegurarse de que se encontró una única fila para cada uno
if gross_profit.shape[0] != 1 or total_revenue.shape[0] != 1:
    raise ValueError("No se encontraron filas únicas para 'grossProfit' o 'totalRevenue'")

# Convertir las columnas a tipo float excluyendo la primera columna (que contiene el nombre de las filas)
gross_profit_values = gross_profit.iloc[0, 1:].astype(float).values
total_revenue_values = total_revenue.iloc[0, 1:].astype(float).values

# Calcular el Ratio de Margen Bruto
ratio_margen_bruto = (gross_profit_values / total_revenue_values) * 100

# Crear un DataFrame para la fecha y el Ratio de Margen Bruto
ratio_margen_bruto_df = pd.DataFrame({
    'Fecha': gross_profit.columns[1:],  # Excluyendo la primera columna
    'Ratio de Margen Bruto': ratio_margen_bruto
})

# Convertir la columna 'Fecha' a tipo datetime
ratio_margen_bruto_df['Fecha'] = pd.to_datetime(ratio_margen_bruto_df['Fecha'])

# Establecer 'Fecha' como índice en ratio_margen_bruto_df
ratio_margen_bruto_df.set_index('Fecha', inplace=True)

# Crear el DataFrame rentabilidad_df
rentabilidad_df = ratio_margen_bruto_df.copy()

# Convertir los valores de 'Ratio de Margen Bruto' a float sin el formato de porcentaje
rentabilidad_df['Ratio de Margen Bruto'] = rentabilidad_df['Ratio de Margen Bruto'].astype(float)

MARGEN OPERATIVO

In [16]:
# Buscar las filas correspondientes a 'operatingIncome' y 'totalRevenue' en la primera columna
operating_income = income_statement_df[income_statement_df.iloc[:, 0] == 'operatingIncome']
total_revenue = income_statement_df[income_statement_df.iloc[:, 0] == 'totalRevenue']

# Asegurarse de que se encontró una única fila para cada uno
if operating_income.shape[0] != 1 or total_revenue.shape[0] != 1:
    raise ValueError("No se encontraron filas únicas para 'operatingIncome' o 'totalRevenue'")

# Convertir las columnas a tipo float excluyendo la primera columna (que contiene el nombre de las filas)
operating_income_values = operating_income.iloc[0, 1:].astype(float).values
total_revenue_values = total_revenue.iloc[0, 1:].astype(float).values

# Calcular el Ratio de Margen Operativo
ratio_margen_operativo = (operating_income_values / total_revenue_values) * 100

# Crear un DataFrame para la fecha y el Ratio de Margen Operativo
ratio_margen_operativo_df = pd.DataFrame({
    'Fecha': operating_income.columns[1:],  # Excluyendo la primera columna
    'Ratio de Margen Operativo': ratio_margen_operativo
})

# Convertir la columna 'Fecha' a tipo datetime
ratio_margen_operativo_df['Fecha'] = pd.to_datetime(ratio_margen_operativo_df['Fecha'])

# Establecer 'Fecha' como índice en ratio_margen_operativo_df
ratio_margen_operativo_df.set_index('Fecha', inplace=True)

# Agregar esta columna a rentabilidad_df
if 'rentabilidad_df' in globals():
    rentabilidad_df = rentabilidad_df.drop(columns=['Ratio de Margen Operativo'], errors='ignore')
    rentabilidad_df = rentabilidad_df.join(ratio_margen_operativo_df, how='outer')
else:
    rentabilidad_df = ratio_margen_operativo_df.copy()

ROA

In [17]:
# Buscar las filas correspondientes a 'totalRevenue' en income_statement_df y 'totalAssets' en balance_sheet_df
total_revenue = income_statement_df[income_statement_df.iloc[:, 0] == 'totalRevenue']
total_assets = balance_sheet_df[balance_sheet_df.iloc[:, 0] == 'totalAssets']

# Asegurarse de que se encontró una única fila para cada uno
if total_revenue.shape[0] != 1 or total_assets.shape[0] != 1:
    raise ValueError("No se encontraron filas únicas para 'totalRevenue' o 'totalAssets'")

# Convertir las columnas a tipo float excluyendo la primera columna (que contiene el nombre de las filas)
total_revenue_values = total_revenue.iloc[0, 1:].astype(float).values
total_assets_values = total_assets.iloc[0, 1:].astype(float).values

# Calcular el Ratio de Rentabilidad de los Activos (ROA)
ratio_rentabilidad_activos = (total_revenue_values / total_assets_values) * 100

# Crear un DataFrame para la fecha y el Ratio de Rentabilidad de los Activos (ROA)
ratio_rentabilidad_activos_df = pd.DataFrame({
    'Fecha': total_revenue.columns[1:],  # Excluyendo la primera columna
    'ROA': ratio_rentabilidad_activos
})

# Convertir la columna 'Fecha' a tipo datetime
ratio_rentabilidad_activos_df['Fecha'] = pd.to_datetime(ratio_rentabilidad_activos_df['Fecha'])

# Establecer 'Fecha' como índice en ratio_rentabilidad_activos_df
ratio_rentabilidad_activos_df.set_index('Fecha', inplace=True)

# Agregar esta columna a rentabilidad_df
if 'rentabilidad_df' in globals():
    rentabilidad_df = rentabilidad_df.drop(columns=['ROA'], errors='ignore')
    rentabilidad_df = rentabilidad_df.join(ratio_rentabilidad_activos_df, how='outer')
else:
    rentabilidad_df = ratio_rentabilidad_activos_df.copy()

ROE

In [18]:
# Buscar las filas correspondientes a 'netIncome' en income_statement_df y 'totalShareholderEquity' en balance_sheet_df
net_income = income_statement_df[income_statement_df.iloc[:, 0] == 'netIncome']
total_shareholder_equity = balance_sheet_df[balance_sheet_df.iloc[:, 0] == 'totalShareholderEquity']

# Asegurarse de que se encontró al menos una fila para cada uno
if net_income.shape[0] == 0 or total_shareholder_equity.shape[0] == 0:
    raise ValueError("No se encontraron filas para 'netIncome' o 'totalShareholderEquity'")

# Convertir las columnas a tipo float excluyendo la primera columna (que contiene el nombre de las filas)
net_income_values = net_income.iloc[0, 1:].astype(float).values
total_shareholder_equity_values = total_shareholder_equity.iloc[0, 1:].astype(float).values

# Calcular el Ratio de Rentabilidad sobre Fondos Propios (ROE)
ratio_rentabilidad_fondos_propios = (net_income_values / total_shareholder_equity_values) * 100

# Crear un DataFrame para la fecha y el Ratio de Rentabilidad sobre Fondos Propios (ROE)
ratio_rentabilidad_fondos_propios_df = pd.DataFrame({
    'Fecha': net_income.columns[1:],  # Excluyendo la primera columna
    'ROE': ratio_rentabilidad_fondos_propios
})

# Convertir la columna 'Fecha' a tipo datetime
ratio_rentabilidad_fondos_propios_df['Fecha'] = pd.to_datetime(ratio_rentabilidad_fondos_propios_df['Fecha'])

# Establecer 'Fecha' como índice en ratio_rentabilidad_fondos_propios_df
ratio_rentabilidad_fondos_propios_df.set_index('Fecha', inplace=True)

# Agregar esta columna a rentabilidad_df
if 'rentabilidad_df' in globals():
    rentabilidad_df = rentabilidad_df.drop(columns=['ROE'], errors='ignore')
    rentabilidad_df = rentabilidad_df.join(ratio_rentabilidad_fondos_propios_df, how='outer')
else:
    rentabilidad_df = ratio_rentabilidad_fondos_propios_df.copy()

P/B RATIO

In [19]:
# Renombrar la columna sin nombre en prices_df a 'date'
prices_df.rename(columns={'Unnamed: 0': 'date', 'Close': 'price'}, inplace=True)
prices_df['date'] = pd.to_datetime(prices_df['date'])

# Convertir la columna 'fiscalDateEnding' a tipo datetime en balance_sheet_df y transponer
balance_sheet_df = balance_sheet_df.set_index('fiscalDateEnding').T
balance_sheet_df.index = pd.to_datetime(balance_sheet_df.index)

# Calcular el valor contable por acción
balance_sheet_df['book_value_per_share'] = (
    balance_sheet_df['totalShareholderEquity'] / balance_sheet_df['commonStockSharesOutstanding']
)

# Función para encontrar el precio más cercano a una fecha dada
def find_closest_price(date, prices_df):
    # Calcular la diferencia absoluta de fechas y encontrar la más cercana
    closest_row = prices_df.loc[(prices_df['date'] - date).abs().idxmin()]
    return closest_row['price']

# Aplicar la función para encontrar el precio más cercano para cada fecha en balance_sheet_df
balance_sheet_df['closest_price'] = balance_sheet_df.index.to_series().apply(
    lambda x: find_closest_price(x, prices_df)
)

# Calcular el P/B Ratio
balance_sheet_df['P/B Ratio'] = balance_sheet_df['closest_price'] / balance_sheet_df['book_value_per_share']

# Asegurarse de que las columnas que queremos añadir no existen en rentabilidad_df
rentabilidad_df['P/B Ratio'] = balance_sheet_df['P/B Ratio']

RENTABILIDAD POR DIVIDENDO

In [ ]:
# Renombrar la columna sin nombre en prices_df a 'date'
prices_df.rename(columns={'Unnamed: 0': 'date', 'Close': 'price'}, inplace=True)
prices_df['date'] = pd.to_datetime(prices_df['date'])

# Convertir la columna 'fiscalDateEnding' a tipo datetime en balance_sheet_df
balance_sheet_df.index = pd.to_datetime(balance_sheet_df.index, errors='coerce')

# Convertir las columnas a tipo datetime en cash_flow_df
cash_flow_df.columns = pd.to_datetime(cash_flow_df.columns, errors='coerce')
cash_flow_df.set_index(cash_flow_df.iloc[:, 0], inplace=True)
cash_flow_df.drop(cash_flow_df.columns[0], axis=1, inplace=True)

# Calcular el dividendo por acción
dividendos_por_accion = (
    cash_flow_df.loc['dividendPayoutCommonStock'].astype(float) /
    balance_sheet_df['commonStockSharesOutstanding'].astype(float)
)

# Función para encontrar el precio más cercano a una fecha dada
def find_closest_price(date, prices_df):
    closest_row = prices_df.loc[(prices_df['date'] - date).abs().idxmin()]
    return closest_row['price']

# Aplicar la función para encontrar el precio más cercano para cada fecha en balance_sheet_df
closest_prices = balance_sheet_df.index.to_series().apply(lambda x: find_closest_price(x, prices_df))

# Calcular el ratio dividendo/precio
rendimiento_por_dividendo = dividendos_por_accion / closest_prices.values

# Añadir las nuevas columnas a rentabilidad_df
rentabilidad_df['Dividendos por Acción'] = dividendos_por_accion.values
rentabilidad_df['Rendimiento por Dividendos'] = rendimiento_por_dividendo.values

EPS

In [21]:
# Convertir las columnas a tipo datetime en cash_flow_df y balance_sheet_df
cash_flow_df.columns = pd.to_datetime(cash_flow_df.columns, errors='coerce')
balance_sheet_df.index = pd.to_datetime(balance_sheet_df.index, errors='coerce')

# Extraer netIncome de cash_flow_df
net_income = cash_flow_df.loc['netIncome'].astype(float)

# Calcular las ganancias por acción
earnings_per_share = net_income / balance_sheet_df['commonStockSharesOutstanding'].astype(float)

# Añadir las ganancias por acción a rentabilidad_df, asegurándose de que solo haya una columna llamada EPS
rentabilidad_df['EPS'] = earnings_per_share.values

# Eliminar la columna 'earnings_per_share' si existe
if 'earnings_per_share' in rentabilidad_df.columns:
    rentabilidad_df.drop(columns=['earnings_per_share'], inplace=True)

P/E RATIO

In [22]:
# Renombrar la columna sin nombre en prices_df a 'date'
prices_df.rename(columns={'Unnamed: 0': 'date', 'Close': 'price'}, inplace=True)
prices_df['date'] = pd.to_datetime(prices_df['date'])

# Convertir la columna 'fiscalDateEnding' a tipo datetime en rentabilidad_df
rentabilidad_df.index = pd.to_datetime(rentabilidad_df.index, errors='coerce')

# Función para encontrar el precio más cercano a una fecha dada
def find_closest_price(date, prices_df):
    closest_date = prices_df.iloc[(prices_df['date'] - date).abs().argsort()[:1]]['date'].values[0]
    closest_price = prices_df[prices_df['date'] == closest_date]['price'].values[0]
    return closest_price

# Aplicar la función para encontrar el precio más cercano para cada fecha en rentabilidad_df
rentabilidad_df['closest_price'] = rentabilidad_df.index.to_series().apply(lambda x: find_closest_price(x, prices_df))

# Calcular el P/E Ratio
rentabilidad_df['P/E Ratio'] = rentabilidad_df['closest_price'] / rentabilidad_df['EPS']

# Eliminar la columna 'closest_price'
rentabilidad_df.drop(columns=['closest_price'], inplace=True)

***RATIOS DE RENTABILIDAD Y PRECIO***

In [ ]:
print("DataFrame con el Ratio de Rentabilidad sobre Fondos Propios por Año:")
print(rentabilidad_df)

In [ ]:
# Asegúrate de que rentabilidad_df tiene el índice de tipo datetime
rentabilidad_df.index = pd.to_datetime(rentabilidad_df.index)

# Ordenar el DataFrame por fechas
rentabilidad_df = rentabilidad_df.sort_index()

# Función para graficar cada columna con eje Y comenzando en 0
def graficar_columna(df, columna):
    plt.figure(figsize=(10, 6))
    plt.plot(df.index, df[columna], marker='o')
    plt.title(f'Gráfico de {columna}')
    plt.xlabel('Fecha')
    plt.ylabel(columna)
    plt.ylim(bottom=0)  # Establecer el límite inferior del eje Y a 0
    plt.grid(True)
    plt.show()

# Graficar cada columna del DataFrame
for columna in rentabilidad_df.columns:
    graficar_columna(rentabilidad_df, columna)

***GRÁFICO DE PRECIOS***

In [ ]:
# Asegúrate de que la columna 'date' esté en formato datetime
prices_df['date'] = pd.to_datetime(prices_df['date'])

# Establecer 'date' como índice
prices_df.set_index('date', inplace=True)

# Ordenar el DataFrame por fechas
prices_df = prices_df.sort_index()

# Graficar el DataFrame sin puntos, solo líneas
plt.figure(figsize=(12, 6))
plt.plot(prices_df.index, prices_df['price'])
plt.title('Gráfico de Precios')
plt.xlabel('Fecha')
plt.ylabel('Precio')
plt.ylim(bottom=0)  # Establecer el límite inferior del eje Y a 0
plt.grid(True)
plt.show()